In [7]:
import requests

In [ ]:
clef = 'RGAPI-10062e3a-ad75-4038-9101-786837289f98'

In [11]:
req = requests.get('https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Quoicoubol?api_key=RGAPI-10062e3a-ad75-4038-9101-786837289f98')

In [13]:
player_info = req.json()

In [14]:
player_info

{'id': 'RJ3eF7Nsc_XC2bhQkYEtMpbaojjmQ_FnrKcIrcCmXosAnI37',
 'accountId': 'Tp5IAh29SKAFc6r-xztZsvo04WNOiKF_T921gd9Xr9a0VN7YTq6Oi98A',
 'puuid': 'SOxYYxeBZAQ0GBWryjohVK4HV3MYzEGJhje6ceDOTVZUn7gDLbM6Qts1PwLfsrd0HTlHhTG_aOvY-w',
 'name': 'Quoicoubol',
 'profileIconId': 23,
 'revisionDate': 1685397928159,
 'summonerLevel': 295}

In [15]:
def get_match_id_by_puuid(puuid,api_key):
    return "https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/"+puuid+"/ids?start=0&count=20&api_key="+api_key

In [16]:
def get_match_info_by_id(match_id,api_key):
    return "https://europe.api.riotgames.com/lol/match/v5/matches/"+match_id+"?api_key="+api_key

In [17]:
api_url = "https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_6428154821?api_key=RGAPI-10062e3a-ad75-4038-9101-786837289f98"

In [18]:
resp = requests.get(api_url)
match_data = resp.json()

In [27]:
player_data = match_data['info']['participants'][3]

In [29]:
player_data

{'allInPings': 0,
 'assistMePings': 0,
 'assists': 29,
 'baitPings': 0,
 'baronKills': 0,
 'basicPings': 0,
 'bountyLevel': 0,
 'challenges': {'12AssistStreakCount': 0,
  'abilityUses': 208,
  'acesBefore15Minutes': 2,
  'alliedJungleMonsterKills': 0,
  'baronTakedowns': 0,
  'blastConeOppositeOpponentCount': 0,
  'bountyGold': 1425,
  'buffsStolen': 0,
  'completeSupportQuestInTime': 0,
  'controlWardsPlaced': 0,
  'damagePerMinute': 1490.7601622265968,
  'damageTakenOnTeamPercentage': 0.21949835268380696,
  'dancedWithRiftHerald': 0,
  'deathsByEnemyChamps': 21,
  'dodgeSkillShotsSmallWindow': 13,
  'doubleAces': 0,
  'dragonTakedowns': 0,
  'earlyLaningPhaseGoldExpAdvantage': 0,
  'effectiveHealAndShielding': 0,
  'elderDragonKillsWithOpposingSoul': 0,
  'elderDragonMultikills': 0,
  'enemyChampionImmobilizations': 54,
  'enemyJungleMonsterKills': 0,
  'epicMonsterKillsNearEnemyJungler': 0,
  'epicMonsterKillsWithin30SecondsOfSpawn': 0,
  'epicMonsterSteals': 0,
  'epicMonsterStolen

In [30]:
player_data['summonerName'],player_data['championName']

('Yoyoooo', 'Nunu')